In [1]:
import pandas as pd 
# importer la bib pandas

In [2]:
df = pd.read_csv('C:/Users/Haifa Elhorra/Desktop/ML/Projet_ML/DATAset/data.csv', sep='|')
# Importer le fichier CSV contenant le dataset en spécifiant le séparateur "|" pour lire correctement les colonnes

In [3]:
legit = df['legitimate']
# Extraire la colonne des labels (légitime ou non)

In [4]:
X = df.drop(['Name','md5','legitimate'],axis=1).values
y = legit
# Séparer les caractéristiques (X) et les labels (y)

In [5]:
import sklearn.ensemble as ek # Pour le classificateur Extra Trees
from sklearn.feature_selection import SelectFromModel # Pour la sélection des caractéristiques
extratrees = ek.ExtraTreesClassifier().fit(X,y)  #Le classificateur calcule l'importance de chaque caractéristique.
model = SelectFromModel(extratrees, prefit=True) #Sélectionner les caractéristiques importantes 
X_new = model.transform(X) #Le nouvel ensemble ne contient que les colonnes correspondant aux caractéristiques importantes.

In [6]:
nbfeatures = X_new.shape[1] #Obtenir le nombre de caractéristiques sélectionnées
print(nbfeatures) # Afficher le nombre de caractéristiques sélectionnées

13


In [7]:
import numpy # Importer la bibliothèque numpy
features = [] # Initialiser une liste vide pour stocker les noms des caractéristiques sélectionnées.
index = numpy.argsort(extratrees.feature_importances_)[::-1][:nbfeatures]  
# Trier les indices des caractéristiques par importance (ordre décroissant).
# Conserver uniquement les `13` premières caractéristiques.
for f in range(nbfeatures):  # Boucle pour parcourir les `13` premières caractéristiques importantes.
    print("%d. feature %s (%f)" % (f + 1, df.columns[2+index[f]], extratrees.feature_importances_[index[f]]))
    # Afficher le rang, le nom et l'importance de chaque caractéristique.
    features.append(df.columns[2+f])
    # Ajouter le nom de la caractéristique sélectionnée à la liste `features`.

1. feature DllCharacteristics (0.149589)
2. feature Machine (0.116449)
3. feature Characteristics (0.113833)
4. feature VersionInformationSize (0.070311)
5. feature SectionsMaxEntropy (0.067852)
6. feature Subsystem (0.064490)
7. feature MajorSubsystemVersion (0.061050)
8. feature ResourcesMaxEntropy (0.045909)
9. feature ImageBase (0.044428)
10. feature ResourcesMinEntropy (0.027495)
11. feature MajorOperatingSystemVersion (0.023474)
12. feature SizeOfStackReserve (0.021556)
13. feature SizeOfOptionalHeader (0.018541)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y ,test_size=0.2, stratify = y)
#Indique que 20 % des données doivent être utilisées pour l'ensemble de test, laissant 80 % pour l'entraînement.

In [9]:
from sklearn.tree import DecisionTreeClassifier # Importer la classe pour construire un arbre de décision.
from sklearn.ensemble import RandomForestClassifier # Importer la classe pour construire une forêt aléatoire.

decision_tree = DecisionTreeClassifier(max_depth=10) # Créer un modèle d'arbre de décision avec une profondeur maximale de 10.  
decision_tree.fit(X_train, y_train) # Entraîner l'arbre de décision avec les données d'entraînement.
dt_score = decision_tree.score(X_test, y_test) # Calculer l'exactitude (accuracy) de l'arbre de décision sur les données de test.
print("DecisionTree: %s" % dt_score) # Afficher le score de l'arbre de décision.

random_forest = RandomForestClassifier(n_estimators=50) # Créer un modèle de forêt aléatoire avec 50 arbres.
random_forest.fit(X_train, y_train) # Entraîner la forêt aléatoire avec les données d'entraînement.
rf_score = random_forest.score(X_test, y_test) # Calculer l'exactitude (accuracy) de la forêt aléatoire sur les données de test.
print("RandomForest: %s" % rf_score) # Afficher le score de la forêt aléatoire.

if dt_score > rf_score:     # Comparer les scores des deux modèles.
    print("Using DecisionTree for classification with", len(features), "features.")
        # Si l'arbre de décision a le meilleur score, afficher un message.
    winner = "Decision Tree"
        # Stocker le nom du modèle gagnant.
else:
    print("Using RandomForest for classification with", len(features), "features.")
        # Si la forêt aléatoire a le meilleur score ou est égale.
    winner = "Random Forest" 
        # Stocker le nom du modèle gagnant.


DecisionTree: 0.9906079832142679
RandomForest: 0.9941299895089174
Using RandomForest for classification with 13 features.


In [10]:
import pickle # Importer le module pickle pour sauvegarder des objets Python.
import joblib # Importer le module joblib pour sauvegarder des modèles plus efficacement.

classifier = model # Assigner le modèle entraîné à la variable `classifier`.
features = X_train.columns if hasattr(X_train, 'columns') else [f"feature_{i}" for i in range(len(X_train[0]))]
# Vérifier si `X_train` a des noms de colonnes.  
# Si oui, les utiliser comme noms des caractéristiques. Sinon, générer des noms.
joblib.dump(classifier, r'C:\Users\Haifa Elhorra\Desktop\ML\Projet_ML\version final\classifier.pkl')
# Sauvegarder le modèle `classifier` dans un fichier nommé `classifier.pkl`.
with open(r'C:\Users\Haifa Elhorra\Desktop\ML\Projet_ML\version final\features.pkl', 'wb') as f:
    # Ouvrir un fichier nommé `features.pkl` en mode écriture binaire.  
    pickle.dump(features, f)
    # Sauvegarder la liste des caractéristiques dans ce fichier.
print("Classifier and features saved successfully!") 
# Afficher un message de confirmation une fois les fichiers sauvegardés.

Classifier and features saved successfully!


In [11]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [12]:
import tensorflow as tf # Importation de TensorFlow pour construire le réseau de neurones
from tensorflow.keras.models import Sequential # Importation du modèle séquentiel de Keras (un modèle où les couches sont empilées les unes après les autres)
from tensorflow.keras.layers import Dense, Input # Importation de la couche Dense (couches entièrement connectées du réseau) et Input pour définir nbr des caractéristiques de couche d'entréé

In [13]:
model = Sequential()
# Définir le modèle séquentiel
model.add(Input(shape=(13,)))
# Ajouter une couche d'entrée avec 13 caractéristiques
model.add(Dense(16, activation="relu"))
# Ajout de la première couche Dense avec 16 neurones, activation ReLU
model.add(Dense(8, activation= "relu"))
# Ajout de la deuxième couche Dense avec 8 neurones et activation ReLU
model.add(Dense(4, activation= "relu"))
# Ajout de la troisième couche Dense avec 4 neurones et activation ReLU
model.add(Dense(1, activation='sigmoid'))
# Ajouter une couche de sortie avec une activation sigmoïde pour la classification binaire
model.summary()
# Afficher le résumé du modèle

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16)                  │             224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 401 (1.57 KB)

 Trainable params: 401 (1.57 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss= "binary_crossentropy" , optimizer="rmsprop", metrics=["accuracy"])
# Compilation du modèle avec une fonction de perte binaire, un optimiseur RMSprop, et la précision comme métrique

In [15]:
model.fit(X_train, y_train, epochs=5, batch_size=32)
# Entraînement du modèle sur les données d'entraînement (5 époques, taille de lot de 32)

Epoch 1/5
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.6684 - loss: 134752.7812
Epoch 2/5
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6985 - loss: 0.6427
Epoch 3/5
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7000 - loss: 0.6582
Epoch 4/5
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7006 - loss: 0.6376
Epoch 5/5
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 3s 1000us/step - accuracy: 0.6998 - loss: 0.6340


In [16]:
from sklearn.metrics import accuracy_score 
# Accuracy on the training dataset
trainPred=model.predict(X_train) # Prédiction des probabilités pour les données d'entraînement
trainPred=[1 if y>= 0.5 else 0   for y in trainPred] # Conversion des probabilités en classes binaires (1 si >= 0,5, sinon 0)
accuracy_score(y_train,trainPred) # Calcul de la précision des prédictions sur les données d'entraînement

3063/3063 ━━━━━━━━━━━━━━━━━━━━ 2s 495us/step


0.7006519543325885

In [17]:
# Accuracy on the test dataset
y_prediction=model.predict(X_test) # Prédiction des probabilités pour les données de test
y_prediction=[1 if y>= 0.5 else 0   for y in y_prediction] # Conversion des probabilités en classes binaires pour les données de test
accuracy_score(y_test, y_prediction) # Calcul de la précision des prédictions sur les données de test

1252/1252 ━━━━━━━━━━━━━━━━━━━━ 1s 512us/step


0.7006544437228356

In [39]:
model.save('dl_pe.keras')
# Sauvegarder le modèle entraîné
